In [1]:
import glob
import os
import cv2
import imutils
import argparse
import numpy as np
from PIL import Image
from pathlib import Path
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, ConvLSTM2D, Conv3DTranspose
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import albumentations as A

In [2]:
# Define the augmentation pipeline
augmentation_pipeline = A.Compose([
    A.RandomBrightnessContrast(p=0.5),  # Random brightness and contrast adjustments
    A.RandomShadow(p=0.5),  # Random shadows
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=30, p=0.5),  # Random shifts, scales, and rotations
    A.GaussianBlur(p=0.5),  # Gaussian blur
    A.GaussNoise(p=0.5),  # Gaussian noise
    A.RandomCrop(227, 227),  # Random crop
    A.ElasticTransform(alpha=3, sigma=1, alpha_affine=1, p=0.5)  # Elastic transformations
])

In [3]:
# Function to apply augmentation to an image
def apply_augmentation(image):
    augmented = augmentation_pipeline(image=image)
    augmented_image = augmented['image']
    return augmented_image

In [4]:
# Function to store image in array
def store_inarray(image_path, lst):
    image = load_img(image_path)
    image = img_to_array(image)
    image = apply_augmentation(image)  # Apply augmentation to the image
    gray = 0.2989 * image[:, :, 0] + 0.5870 * image[:, :, 1] + 0.1140 * image[:, :, 2]
    lst.append(gray)

In [5]:
# Paths
train_path = r'D:\python projects\Intelligent_Video_Surveillance\Avenue Dataset\training_videos'
fps = 5
train_images_path = os.path.join(train_path, 'frames')
os.makedirs(train_images_path, exist_ok=True)

In [6]:
train_videos = glob.glob(os.path.join(train_path, '*.avi'))
store_image_train = []

if not os.listdir(train_images_path):
    for video_path in train_videos:
        video_name = os.path.basename(video_path)
        video_folder = os.path.join(train_images_path, video_name[:-4])
        os.makedirs(video_folder, exist_ok=True)

        # Open the video file
        cap = cv2.VideoCapture(video_path)

        frame_count = 0

        # Read frames until the video is complete
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            # Save frame as an image file
            frame_path = os.path.join(video_folder, '{:03d}.jpg'.format(frame_count))
            cv2.imwrite(frame_path, frame)

            store_inarray(frame_path, store_image_train)

            frame_count += 1

        cap.release()
else:
    print("flames are already avalable")
# Print the number of training frames extracted
print("Training frames extracted:", len(store_image_train))


flames are already avalable
Training frames extracted: 0


In [7]:
if len(store_image_train) > 0:
    store_image_train = np.array(store_image_train)
    a, b, c = store_image_train.shape

    store_image_train.resize(b, c, a)
    store_image_train = (store_image_train - store_image_train.mean()) / store_image_train.std()
    store_image_train = np.clip(store_image_train, 0, 1)
    np.save('training.npy', store_image_train)
    print("Training images saved as numpy array.")
else:
    print("No training images found.")


No training images found.


In [8]:
# Load the training and testing datasets
training_data = np.load('./Avenue Dataset/training_videos/frames/training.npy')

In [9]:
print("Training data shape:", training_data.shape)

Training data shape: (227, 227, 14919)


In [10]:
frames_train = training_data.shape[2]
frames_train = frames_train - frames_train % 10
training_data = training_data[:, :, :frames_train]
training_data = training_data.reshape(-1, 227, 227, 10)
training_data = np.expand_dims(training_data, axis=4)

In [11]:
stae_model=Sequential()

stae_model.add(Conv3D(filters=64,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',input_shape=(227,227,10,1),activation='tanh'))
stae_model.add(Conv3D(filters=32,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,padding='same',dropout=0.4,recurrent_dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=32,kernel_size=(3,3),strides=1,padding='same',dropout=0.3,return_sequences=True))
stae_model.add(ConvLSTM2D(filters=64,kernel_size=(3,3),strides=1,return_sequences=True, padding='same',dropout=0.5))
stae_model.add(Conv3DTranspose(filters=128,kernel_size=(5,5,1),strides=(2,2,1),padding='valid',activation='tanh'))
stae_model.add(Conv3DTranspose(filters=1,kernel_size=(11,11,1),strides=(4,4,1),padding='valid',activation='tanh'))

stae_model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [12]:
epochs=3
batch_size=1
callback_save = ModelCheckpoint("model2.h5", monitor="mean_squared_error", save_best_only=True)
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [13]:
target_data=training_data.copy()
stae_model.fit(training_data, target_data, batch_size=batch_size, epochs=epochs, callbacks=[callback_save, callback_early_stopping])

Epoch 1/3
1491/1491 [==============================] - 1406s 939ms/step - loss: 0.0215 - accuracy: 0.9523
Epoch 2/3
1491/1491 [==============================] - 1903s 1s/step - loss: 0.0139 - accuracy: 0.9618
Epoch 3/3
1491/1491 [==============================] - 2324s 2s/step - loss: 0.0125 - accuracy: 0.9634


In [14]:
stae_model.save("model2.h5")

In [15]:
stae_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 55, 55, 10, 64)    7808      
                                                                 
 conv3d_1 (Conv3D)           (None, 26, 26, 10, 32)    51232     
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 26, 26, 10, 64)    221440    
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 26, 26, 10, 32)    110720    
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 26, 26, 10, 64)    221440    
                                                                 
 conv3d_transpose (Conv3DTra  (None, 55, 55, 10, 128)  204928    
 nspose)                                                         
                                                        

In [1]:
import cv2
import numpy as np
from keras.models import load_model
import imutils

def mean_squared_loss(x1, x2):
    difference = x1 - x2
    a, b, c, d, e = difference.shape
    n_samples = a * b * c * d * e
    sq_difference = difference ** 2
    Sum = sq_difference.sum()
    distance = np.sqrt(Sum)
    mean_distance = distance / n_samples
    return mean_distance

model = load_model("saved_model.h5")

path = './test/4.mp4'
cap = cv2.VideoCapture(path)
print(cap.isOpened())

mse_scores = []  # Store MSE scores for anomaly detection
threshold = 0.0007  # Initial threshold value
alpha = 0.1  # Smoothing factor for temporal smoothing

while cap.isOpened():
    imagedump = []
    ret, frame = cap.read()

    if frame is None:
        break

    for i in range(10):
        ret, frame = cap.read()
        if frame is None:
            break

        image = imutils.resize(frame, width=700, height=600)
        frame = cv2.resize(frame, (227, 227), interpolation=cv2.INTER_AREA)
        gray = 0.2989 * frame[:, :, 0] + 0.5870 * frame[:, :, 1] + 0.1140 * frame[:, :, 2]
        gray = (gray - gray.mean()) / gray.std()
        gray = np.clip(gray, 0, 1)
        imagedump.append(gray)

    if frame is None:
        break

    imagedump = np.array(imagedump)
    imagedump.resize(227, 227, 10)
    imagedump = np.expand_dims(imagedump, axis=0)
    imagedump = np.expand_dims(imagedump, axis=4)

    output = model.predict(imagedump)
    reconstructed_frames = np.clip(output, 0, 1)

    loss = mean_squared_loss(imagedump, reconstructed_frames)
    mse_scores.append(loss)
    print(loss)
    if frame.any() is None:
        print("none")

    # Temporal smoothing
    if len(mse_scores) > 1:
        smoothed_loss = alpha * loss + (1 - alpha) * mse_scores[-2]
        mse_scores[-1] = smoothed_loss
    else:
        smoothed_loss = loss

    # Adjust the threshold dynamically based on past scores
    if len(mse_scores) > 10:
        previous_scores = mse_scores[-10:-1]
        mean_score = np.mean(previous_scores)
        std_score = np.std(previous_scores)
        threshold = mean_score + 2 * std_score  # Adjust the threshold as per your requirement

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

    if smoothed_loss > threshold:
        print('Abnormal Event Detected')
        cv2.putText(image, "Abnormal Event", (100, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4)

    cv2.imshow("video", image)

cap.release()
cv2.destroyAllWindows()

True
1/1 [==============================] - 1s 1s/step
0.0006968945126440364
1/1 [==============================] - 0s 258ms/step
0.0006968593140592546
1/1 [==============================] - 0s 238ms/step
0.0006968894451003911
1/1 [==============================] - 0s 235ms/step
0.0006960997772446992
1/1 [==============================] - 0s 217ms/step
0.0007073405982375892
1/1 [==============================] - 0s 226ms/step
0.0007096554525229898
1/1 [==============================] - 0s 211ms/step
0.000727454213594237
Abnormal Event Detected
1/1 [==============================] - 0s 246ms/step
0.0007441157609705694
Abnormal Event Detected
1/1 [=======================